In [1]:
from sbalign.training.diffusivity import *
from torch.distributions.multivariate_normal import MultivariateNormal
import matplotlib.pyplot as plt

In [2]:
a = 1.0 #start value z_0 of process
b = 10.0 #end value z_T of process

# so far it only works for K<=2, where for K=2 it only works for H=>0.5 - other K and H result in non positiv definit covariance matrix
# todo: check all formulars again

H=0.5
K=3

fbb = FBB(H=H, K=K, gamma_min=1.0,gamma_max=20)

N = 1000
T = 1.0
dt = T/N 

time = torch.linspace(0,1,N+1)
Z = torch.ones(N+1,K+1) * a
zT = torch.ones(K+1)*b

for i in range(N):
    z = Z[i]
    t = time[i+1]
    mean,cov = fbb.pinned_statistics(torch.tensor([t]),z,zT)
    mvn = MultivariateNormal(mean,cov)
    sample = mvn.sample(sample_shape=(1,))
    Z[i+1] = sample

plt.plot(time,Z[:,0],label='X')
plt.plot(time,(torch.ones_like(Z)*a)[:,0],label=r'$x_0$')
plt.plot(time,(torch.ones_like(Z)*b)[:,0],label=r'$x_T$')
plt.legend()
plt.show()
plt.close()

plt.plot(time,Z[:,0],label='X')
for k in range(K):
    plt.plot(time,Z[:,k+1], label=f'Y{k}')
plt.plot(time,(torch.ones_like(Z)*b)[:,0],label=r'$z_T$')
plt.legend()
plt.show()
plt.close()

lam tensor([ -0.8491,  -1.0000,  -4.4721, -20.0000])
lam tensor([ -0.8491,  -1.0000,  -4.4721, -20.0000])
lam tensor([ -0.8491,  -1.0000,  -4.4721, -20.0000])
lam tensor([ -0.8491,  -1.0000,  -4.4721, -20.0000])
lam tensor([ -0.8491,  -1.0000,  -4.4721, -20.0000])
lam tensor([ -0.8491,  -1.0000,  -4.4721, -20.0000])


ValueError: Expected parameter covariance_matrix (Tensor of shape (1, 4, 4)) of distribution MultivariateNormal(loc: torch.Size([1, 4]), covariance_matrix: torch.Size([1, 4, 4])) to satisfy the constraint PositiveDefinite(), but found invalid values:
tensor([[[0.0123, 0.0020, 0.0020, 0.0020],
         [0.0020, 0.0117, 0.0018, 0.0017],
         [0.0020, 0.0018, 0.0116, 0.0017],
         [0.0020, 0.0017, 0.0017, 0.0109]]])